In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
import pickle
import os

from PF_Aso1_0_non_parallel import *

workdir = os.path.dirname(os.getcwd())
srcdir = os.getcwd()
datadir = workdir + '/data/'
outputdir = workdir + '/output/'

In [2]:
seed = 7

obs_series = pd.read_csv(datadir + 'data.csv', delimiter=',')
obs_series = np.array(obs_series.iloc[:,1:]).T

T = obs_series.shape[1]
N = 1000
Λ_scale = 1
cd_scale = 1

case = 'actual data, seed = ' + str(seed) + ', T = ' + str(T) + ', N = ' + str(N) + ', Λ_scale = ' + str(Λ_scale) + ', cd_scale = ' + str(cd_scale)
try: 
    casedir = outputdir + case  + '/'
    os.mkdir(casedir)
except:
    casedir = outputdir + case  + '/'

In [3]:
np.random.seed(seed)
start_time = time.time()

D_0 = obs_series[:,[0]]
θ_t_particle = []
X_t_particle = []
H_t_particle = []
for i in range(N):
    θ_0, X_0, H_0, _ = init(D_0, Λ_scale, cd_scale)
    θ_t_particle.append(θ_0)
    X_t_particle.append(X_0)
    H_t_particle.append(H_0)

with open(casedir + 'θ_0.pkl', 'wb') as f:
    pickle.dump(θ_t_particle, f)
with open(casedir + 'X_0.pkl', 'wb') as f:
    pickle.dump(X_t_particle, f)
with open(casedir + 'H_0.pkl', 'wb') as f:
    pickle.dump(H_t_particle, f)
with open(casedir + 'count_0.pkl', 'wb') as f:
    pickle.dump(list(np.ones(N)), f)
with open(casedir + 'w_0.pkl', 'wb') as f:
    pickle.dump(list(np.ones(N)/N), f)
run_time = time.time() - start_time
print(run_time)    

del(θ_t_particle)

for t in tqdm(range(T-1)):
    
    D_t_next = obs_series[:,[t+1]]
    
    θ_t_next_particle = []
    X_t_next_particle = []
    H_t_next_particle = []
    ν_t_next_particle = []
    
    for i in range(N):
        
        θ_t_next = update_θ(H_t_particle[i])
        θ_t_next_particle.append(θ_t_next)

        X_t_next = update_X(D_t_next, X_t_particle[i], θ_t_next)
        X_t_next_particle.append(X_t_next)

        H_t_next = update_H(X_t_next, X_t_particle[i], H_t_particle[i])
        H_t_next_particle.append(H_t_next)

        ν_t_next = update_ν(D_t_next, X_t_particle[i], θ_t_next)
        ν_t_next_particle.append(ν_t_next)
    
    with open(casedir + 'θ_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(θ_t_next_particle, f)
    with open(casedir + 'X_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(X_t_next_particle, f)
    with open(casedir + 'H_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(H_t_next_particle, f)
    
    w_t_next = ν_t_next_particle/np.sum(ν_t_next_particle)
    try:
        count_all = sp.stats.multinomial.rvs(N, w_t_next)
    except:
        for i in range(w_t_next.shape[0]):
            if w_t_next[i]>(np.sum(w_t_next[:-1]) - 1):
                w_t_next[i] = w_t_next[i] - (np.sum(w[:-1]) - 1)
                break
        count_all = sp.stats.multinomial.rvs(N, w_t_next)
    
    with open(casedir + 'w_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(w_t_next, f)
    with open(casedir + 'count_' + str(t+1) + '.pkl', 'wb') as f:
        pickle.dump(count_all, f)
    
    del(X_t_particle)
    del(H_t_particle)
    del(w_t_next)
    del(D_t_next)
    del(θ_t_next_particle)
    del(ν_t_next_particle)
    
    X_t_particle = []
    H_t_particle = []
    
    for i in range(N):
        if count_all[i] != 0:
            for n in range(count_all[i]):
                X_t_particle.append(X_t_next_particle[i])
                H_t_particle.append(H_t_next_particle[i])
    
    del(count_all)            
    del(X_t_next_particle)
    del(H_t_next_particle)
    

9.15268611907959


  0%|                                                   | 0/282 [00:00<?, ?it/s]


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function matmul>) found for signature:
 
 >>> matmul(array(float64, 2d, C), array(int64, 2d, C))
 
There are 2 candidate implementations:
[1m   - Of which 2 did not match due to:
   Overload in function 'MatMul.generic': File: numba/core/typing/npydecl.py: Line 1022.
     With argument(s): '(array(float64, 2d, C), array(int64, 2d, C))':[0m
[1m    Rejected as the implementation raised a specific error:
      TypingError: [1m'@' arguments must all have the same dtype[0m[0m
  raised from /opt/anaconda3/lib/python3.8/site-packages/numba/core/typing/npydecl.py:967
[0m
[0m[1mDuring: typing of intrinsic-call at /Users/EDBC/Dropbox/Mac/Documents/Estimation Projects/44th Version, PF large parallel/src/PF_Aso1_0_non_parallel.py (209)[0m
[1m
File "PF_Aso1_0_non_parallel.py", line 209:[0m
[1mdef bayes_para_update(bt, Λt, ct, dt, Rt_next, Zt_next):
    <sourc